In [1]:
# Importing needed packages

import pandas as pd
import numpy as np
import scipy
from itertools import islice
from collections import Counter
import matplotlib.pyplot as plt
import csv
import math
import warnings
from sklearn.linear_model import LinearRegression


In [2]:
# Read in the dataframe:

all_matches = pd.read_csv('all_matches.csv')
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (33,59,74) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Dimensions of the dataset (#rows, #columns)

all_matches.shape

(249660, 124)

Create a new column for the four groups:

("PG", "WH", "PR", "PT", "P4", "PH") ~ "Left"

("P3", "P2", "P1", "PX", "PC", "PZ") ~ "Middle"

("PA", "PD", "WF", "PB", "PF") ~ "Right"

("VB", "VN", "VM", "WM", "WN") ~ "Back Row"ed:

In [5]:
all_matches['type_group'] = ""

In [6]:
L = ["PG", "WH", "PR", "PT", "P4", "PH"]
M = ["P3", "P2", "P1", "PX", "PC", "PZ"]
R = ["PA", "PD", "WF", "PB", "PF"]
BR = ["VB", "VN", "VM", "WM", "WN"]


# This for-loop goes through the 'attack_code' column and groups them into one of four groups.
for index, row in all_matches.iterrows():
    if row['attack_code'] in L:
        all_matches.at[index, 'type_group'] = 'L'
    elif row['attack_code'] in M:
        all_matches.at[index, 'type_group'] = 'M'
    elif row['attack_code'] in R:
        all_matches.at[index, 'type_group'] = 'R'
    elif row['attack_code'] in BR:
        all_matches.at[index, 'type_group'] = 'BR'
    
    else:
        all_matches.at[index, 'type_group'] = 'NaN'
        

In [8]:
# How many teams total are in this dataset

teams = all_matches['team'].nunique()
print(teams,'teams')

51 teams


In [9]:
# A dictionary that shows how many matches each team has played

unique_vals = dict(all_matches.groupby('team')['match_id'].nunique())
print(unique_vals)

{'AZ': 10, 'AZ St': 10, 'Abilene Christian': 1, 'Albany': 1, 'Baylor': 1, 'CO St': 6, 'CU': 153, 'Cal': 9, 'Campbell': 1, 'DU': 1, 'Den': 1, 'FIU': 1, 'GA': 1, 'IA': 4, 'IL': 5, 'IN St': 1, 'JMU': 1, 'LIU': 1, 'NC': 1, 'NE': 1, 'NSULA': 1, 'OK': 2, 'OR': 9, 'OR St': 9, 'ORU': 1, 'Oakland': 1, 'PA St': 1, 'Portland St': 1, 'Rice': 1, 'SC': 1, 'SJSU': 1, 'Seton Hall': 1, 'Stan': 7, 'TTU': 1, 'UALR': 1, 'UCLA': 9, 'UCSB': 1, 'UMBC': 1, 'UNCC': 1, 'UNCO': 1, 'UNF': 1, 'USC': 9, 'UT': 10, 'UWGB': 1, 'W MI': 1, 'WA': 9, 'WA St': 8, 'WSU': 1, 'WY': 2, 'Wake': 1, 'Xavier': 2}


In [10]:
# Which teams have played only 1 match vs mult. matches

one_match = []
mult_match = []

for key, value in unique_vals.items():
    if value == 1:
        one_match.append(key)
    else:
        mult_match.append(key)

Outside hitter should be identified by hitting a high percentage of sets that fall into the 'Left' (L in type_group) group.

- Make a dataframe of only `skill` = `attack`

- Add up their 'left' attacks and divide it by all the attacks the player has hit

In [11]:
# Grouping the original dataframe 'all_matches' into only the rows with the column skill = attack

attack_only = all_matches[all_matches['skill'] == 'Attack']


left_percentages is a dictionary with the keys being player names and the values being their percentage of attacks that fall into left category.

In [12]:
left_percentages = {}

for player in attack_only['player_name'].unique():
    player_data = attack_only[attack_only['player_name'] == player]
    play_left = player_data['type_group'].value_counts().get('L', 0)
    tot_p = player_data['type_group'].apply(lambda x: ((x=='L') | (x=='R'))| (x=='M') | (x=='BR')).sum().sum()
    player_percent = play_left/tot_p
    left_percentages[player] = player_percent
    
#print(left_percentages)


In [13]:
# How many players in the `left_percentages` dictionary

len(left_percentages)

594

In [14]:
# Getting rid of nan values in left_percentages dictionary

left_percentages = {key: value for key, value in left_percentages.items() if not math.isnan(value)}


In [15]:
# How many players in `left_percentages` dictionary that are not nan

len(left_percentages)

515

In [16]:
# Sorting the `left_percentages` dictionary by the values in ascending order 

left_percentages2 = dict(sorted(left_percentages.items(), key = lambda item: item[1]))

The next few code blocks is making a csv file with the total sum of attacks each player had, and setting a threshold for OH with 11 or more attack attempts

In [17]:
percent_attacks = {}

for player, percent in left_percentages2.items():
    player_df = all_matches[all_matches['player_name'] == player]
    
    sum_attack = (player_df['skill'] == 'Attack').sum()
    
    percent_attacks[player] = sum_attack

In [18]:
#percent_attacks

In [19]:
len(percent_attacks)

515

In [20]:
sum_attacks = {k: v for k, v in percent_attacks.items() if v >= 11}
#sum_attacks

In [21]:
len(sum_attacks)

397

In [22]:
num_attacks = pd.DataFrame.from_dict(percent_attacks, orient = 'index', columns = ['Total Attacks'] )


In [23]:
num_attacks.to_csv('num_attacks', index=True, sep='\t')

In [24]:
sum_attacks2 = pd.DataFrame.from_dict(percent_attacks, orient = 'index', columns = ['Total Attacks'] )
sum_attacks2.to_csv('sum_of_attacks2', index=True, sep='\t')


Making a dictionary witht o 50% threshold and the 11 attack threshold to get the list of OH

In [25]:
#left_percentages2

In [26]:
#percent_attacks

In [27]:
outside_hitters2 = {}

In [28]:
for key, value in percent_attacks.items():
    if value >= 11:
        outside_hitters2[key] = value

In [29]:
#outside_hitters2

In [30]:
# How many outside hitters are in this dictionary
len(outside_hitters2)

397

In [31]:
outside_hitters3 = {}

In [32]:
for key, value in left_percentages2.items():
    if value >= .50:
        outside_hitters3[key] = value

In [33]:
#outside_hitters3

In [34]:

len(outside_hitters3)

197

In [35]:
keys1 = set(outside_hitters2.keys())

In [36]:
keys2 = set(outside_hitters3.keys())

In [37]:
# Creating a list of OH with both thresholds
Outside_Hitters = list(keys1.intersection(keys2))

In [38]:
#Outside_Hitters

In [39]:
len(Outside_Hitters) 

167

In [40]:
# Resetting the 'player_position' column so every value is NA
all_matches['player_position'] = 'NA'

In [41]:
# Labeling the outside hitters in the original dataset

all_matches.loc[all_matches['player_name'].isin(Outside_Hitters), 'player_position'] = 'OH'


Computing the percentage of attacks responsible by the outside hitter for each team:

In [43]:
# A dataframe with attacks as the only skill

attack_df = all_matches[all_matches['skill'] == 'Attack']

In [44]:
# Create a dataframe for only the teams that have played one match:

one_match_df = attack_df[attack_df['team'].isin(one_match)]

In [45]:
# Create a dataframe for the teams that have played more than one match:

mult_match_df = attack_df[attack_df['team'].isin(mult_match)]
teams1 = mult_match_df['team'].unique()
matches1 = mult_match_df['match_id'].unique()

In [46]:
# Calculate the percent of attacks that OH are responsible for in a loop:


attack_percentages2 = {}

for team in teams1:
    
    group = mult_match_df[mult_match_df['team'] == team]
    
    for match in group['match_id'].unique():
        
        group2 = group[group['match_id'] == match]
        
        percent_OH = (group2['player_position'] == 'OH').sum()
        
        tot_percent = (group2['player_position'] == 'OH').sum() + (group2['player_position'] == 'NA').sum()
        
        if tot_percent == 0:
            attack_perc = 0
        else:
            attack_perc = percent_OH / tot_percent
            
        attack_percentages2[(team, match)] = attack_perc
        

In [47]:
df2 = pd.DataFrame.from_dict(attack_percentages2, orient = 'index', columns = ['Percent attacks by OH'] )
print(df2)

                                           Percent attacks by OH
(CU, 24fe712b9d553f6a88347598f84a2616)                  0.585366
(CU, 3ea1d34d13816201660623c1f134edc4)                  0.553571
(CU, 269892222ba8ea67a10ffaf738c1ec28)                  0.488550
(CU, 531ede0ec4ba713908b77173ad24fb51)                  0.526316
(CU, 8ed2dcea382d968235d76653195c5c28)                  0.551020
...                                                          ...
(OR St, e49e03d1200e03589a0c88360b474504)               0.540541
(OK, a68b1199c4dfec9273aa2481b8886cfb)                  0.504000
(OK, 70a851bb68ac3e4e369ef17cabdb5399)                  0.366972
(WY, 3b81cf188d02af7deb4bd99f44bf1698)                  0.369128
(WY, 638ef8538229b68f35389f6031ee7a29)                  0.365385

[273 rows x 1 columns]


Compute the number of receptions made by the outside hitter:

In [48]:
# Create a dataframe only for receptions as the skill:

receps_df = all_matches[all_matches['skill'] == 'Reception']

First do it for the schools that have only played one match:

In [49]:
# Create a dataframe for these schools:

one_match_df2 = receps_df[receps_df['team'].isin(one_match)]

In [50]:
# Calculate the percent of receptions that OH are responsible for in a loop:

receps_percentages_OH = {}

for team in one_match_df2['team'].unique():
    team_df = one_match_df2[one_match_df2['team'] == team]
    percent_OH = (team_df['player_position'] == 'OH').sum()
    tot_percent = (team_df['player_position'] == 'OH').sum() + (team_df['player_position'] == 'S').sum() + (team_df['player_position'] == 'Missing').sum()
    #tot_percent = len(team_df['player_position'])
    attack_per = percent_OH/tot_percent
    receps_percentages_OH[team] = attack_per


In [51]:
df4 = pd.DataFrame.from_dict(receps_percentages_OH, orient = 'index', columns = ['Percent receptions by OH'] )
print(df4)

                   Percent receptions by OH
NSULA                                   1.0
Albany                                  1.0
UNCC                                    1.0
SJSU                                    1.0
TTU                                     1.0
UNCO                                    1.0
UWGB                                    1.0
W MI                                    1.0
Den                                     1.0
FIU                                     1.0
UALR                                    1.0
WSU                                     1.0
Seton Hall                              1.0
JMU                                     1.0
Baylor                                  1.0
NE                                      1.0
ORU                                     1.0
UNF                                     1.0
Rice                                    1.0
GA                                      1.0
NC                                      1.0
Portland St                     

Now do it for the schools that have played more than one match:

In [52]:
# Create a dataframe for schools with multiple matches and skill being receptions
# Create a list of teams with multiple matches
# Create a list of match-ids for each school that have played multiple matches

mult_match_df2 = receps_df[receps_df['team'].isin(mult_match)]
teams2 = mult_match_df2['team'].unique()
matches2 = mult_match_df['match_id'].unique()

In [53]:
# Calculate the percent of receptions that OH are responsible for in a loop:


receps_percentages2 = {}

for team in teams2:
    
    group = mult_match_df2[mult_match_df2['team'] == team]
    
    for match in group['match_id'].unique():
        
        group2 = group[group['match_id'] == match]
        
        percent_OH = (group2['player_position'] == 'OH').sum()
        
        tot_percent = (group2['player_position'] == 'OH').sum() + (group2['player_position'] == 'S').sum() + (group2['player_position'] == 'Missing').sum()
        
        if tot_percent == 0:
            receps_perc = 0
        else:
            receps_perc = percent_OH / tot_percent
            
        receps_percentages2[(team, match)] = receps_perc
        

In [54]:
df5 = pd.DataFrame.from_dict(receps_percentages2, orient = 'index', columns = ['Percent receptions by OH'] )
print(df5)

                                           Percent receptions by OH
(CU, 24fe712b9d553f6a88347598f84a2616)                          1.0
(CU, 3ea1d34d13816201660623c1f134edc4)                          1.0
(CU, 269892222ba8ea67a10ffaf738c1ec28)                          1.0
(CU, 531ede0ec4ba713908b77173ad24fb51)                          1.0
(CU, 8ed2dcea382d968235d76653195c5c28)                          1.0
...                                                             ...
(OR St, e49e03d1200e03589a0c88360b474504)                       1.0
(OK, a68b1199c4dfec9273aa2481b8886cfb)                          1.0
(OK, 70a851bb68ac3e4e369ef17cabdb5399)                          1.0
(WY, 3b81cf188d02af7deb4bd99f44bf1698)                          1.0
(WY, 638ef8538229b68f35389f6031ee7a29)                          1.0

[273 rows x 1 columns]


In [55]:
# Put both dictionaries together into one for all reception percentages for each match:

reception_percentages = receps_percentages_OH.update(receps_percentages2)

In [56]:
#receps_percentages_OH

In [57]:
OH_receps_matches= pd.DataFrame.from_dict(receps_percentages_OH, orient = 'index', columns = ['Percent of receptions made by OH'] )
print(OH_receps_matches)


                                           Percent of receptions made by OH
NSULA                                                                   1.0
Albany                                                                  1.0
UNCC                                                                    1.0
SJSU                                                                    1.0
TTU                                                                     1.0
...                                                                     ...
(OR St, e49e03d1200e03589a0c88360b474504)                               1.0
(OK, a68b1199c4dfec9273aa2481b8886cfb)                                  1.0
(OK, 70a851bb68ac3e4e369ef17cabdb5399)                                  1.0
(WY, 3b81cf188d02af7deb4bd99f44bf1698)                                  1.0
(WY, 638ef8538229b68f35389f6031ee7a29)                                  1.0

[306 rows x 1 columns]


In [58]:
OH_receps_matches.to_csv('Perc_receps_match', index=True, sep='\t')

Compute the number of attacks and receptions made by the outside hitter for each match:

In [59]:
# Create a dataframe with skill only being attacks and receptions:

receps_attacks = all_matches.loc[all_matches['skill'].isin(['Attack', 'Reception'])]

In [60]:
# Start with the teams that have only played one match:

one_match_df2 = receps_attacks[receps_attacks['team'].isin(one_match)]

In [61]:
# Calculate the percent of attacks plus receptions that OH are responsible for in a loop:

attack_and_receps = {}

for team in one_match_df2['team'].unique():
    team_df = one_match_df2[one_match_df2['team'] == team]
    percent_OH = (team_df['player_position'] == 'OH').sum()
    tot_percent = (team_df['player_position'] == 'OH').sum() + (team_df['player_position'] == 'S').sum() + (team_df['player_position'] == 'Missing').sum()
    #tot_percent = len(team_df['player_position'])
    attack_receps = percent_OH/tot_percent
    attack_and_receps[team] = attack_receps
    
    

In [62]:
df6 = pd.DataFrame.from_dict(attack_and_receps, orient = 'index', columns = ['Percent attacks & receptions by OH'] )
print(df6)

                   Percent attacks & receptions by OH
NSULA                                             1.0
Albany                                            1.0
UNCC                                              1.0
SJSU                                              1.0
TTU                                               1.0
UNCO                                              1.0
UWGB                                              1.0
W MI                                              1.0
Den                                               1.0
FIU                                               1.0
UALR                                              1.0
WSU                                               1.0
Seton Hall                                        1.0
JMU                                               1.0
Baylor                                            1.0
NE                                                1.0
ORU                                               1.0
UNF                         

In [63]:
# Create dataframe for teams with multiple matches

mult_match_df2 = receps_attacks[receps_attacks['team'].isin(mult_match)]

In [64]:
# Calculate the percent of attacks plus receptions that OH are responsible for in a loop:
teams2 = mult_match_df2['team'].unique()


attacks_receps2 = {}

for team in teams2:
    
    group = mult_match_df2[mult_match_df2['team'] == team]
    
    for match in group['match_id'].unique():
        
        group2 = group[group['match_id'] == match]
        
        percent_OH = (group2['player_position'] == 'OH').sum()
        
        tot_percent = (group2['player_position'] == 'OH').sum() + (group2['player_position'] == 'S').sum() + (group2['player_position'] == 'Missing').sum()
        
        if tot_percent == 0:
            re_at = 0
        else:
            re_at = percent_OH / tot_percent
            
        attacks_receps2[(team, match)] = re_at
        

In [65]:
df7 = pd.DataFrame.from_dict(attacks_receps2, orient = 'index', columns = ['Percent attacks & receptions by OH'] )
print(df7)


                                           Percent attacks & receptions by OH
(CU, 24fe712b9d553f6a88347598f84a2616)                                    1.0
(CU, 3ea1d34d13816201660623c1f134edc4)                                    1.0
(CU, 269892222ba8ea67a10ffaf738c1ec28)                                    1.0
(CU, 531ede0ec4ba713908b77173ad24fb51)                                    1.0
(CU, 8ed2dcea382d968235d76653195c5c28)                                    1.0
...                                                                       ...
(OR St, e49e03d1200e03589a0c88360b474504)                                 1.0
(OK, a68b1199c4dfec9273aa2481b8886cfb)                                    1.0
(OK, 70a851bb68ac3e4e369ef17cabdb5399)                                    1.0
(WY, 3b81cf188d02af7deb4bd99f44bf1698)                                    1.0
(WY, 638ef8538229b68f35389f6031ee7a29)                                    1.0

[273 rows x 1 columns]


Compute the percentages of matches won by each team:

In [66]:
# First group the data by team: then count how many matches each team won

perc_won = {}

for team in all_matches['team'].unique():
    team_df = all_matches[all_matches['team'] == team]
    percent_won = (team_df['won_match'] == True).sum()
    tot_percent = (team_df['won_match'] == True).sum() + (team_df['won_match'] == False).sum() + (team_df['won_match'] == 'nan').sum()
    
    if tot_percent == 0:
        won_matches = 0
    else:
        won_matches = percent_won/tot_percent
 
    perc_won[team] = won_matches


In [67]:
df8 = pd.DataFrame.from_dict(perc_won, orient = 'index', columns = ['Percent of Matches Won'] )
print(df8)

                   Percent of Matches Won
CU                               0.544543
NSULA                            0.000000
NaN                              0.000000
CO St                            0.294071
Cal                              0.166301
WA St                            0.898724
WA                               0.785216
OR                               0.566902
UCLA                             0.657139
Albany                           0.000000
AZ                               0.698884
UT                               0.673881
AZ St                            0.347155
USC                              0.811503
UNCC                             0.000000
Xavier                           0.000000
IA                               0.000000
IL                               0.162069
SJSU                             0.000000
TTU                              0.000000
UNCO                             0.000000
Stan                             1.000000
OR St                            0

The next few code block are checking that the loop correctly works

In [68]:
den = all_matches[all_matches['team'] == 'Den']
print(den['won_match'].unique())

[False]


In [69]:
DU = all_matches[all_matches['team'] == 'DU']
print(DU['won_match'].unique())

[False]


In [70]:
UMBC = all_matches[all_matches['team'] == 'UMBC']
print(UMBC['won_match'].unique())

[False]


In [71]:
Campbell = all_matches[all_matches['team'] == 'Campbell']
print(Campbell['won_match'].unique())

[False]


In [72]:
Stan = all_matches[all_matches['team'] == 'Stan']
print(Stan['won_match'].unique())

[True]


In [73]:
CU = all_matches[all_matches['team'] == 'CU']
print(CU['won_match'].unique())

[True False]


The next few code blocks are making a csv file with the number of matches each team plays

In [74]:
# Unique teams with the number of matches that they played

unique_vals = dict(all_matches.groupby('team')['match_id'].nunique())
u_v = pd.DataFrame.from_dict(unique_vals, orient = 'index', columns = ['Number of Matches Played'] )
print(u_v)


                   Number of Matches Played
AZ                                       10
AZ St                                    10
Abilene Christian                         1
Albany                                    1
Baylor                                    1
CO St                                     6
CU                                      153
Cal                                       9
Campbell                                  1
DU                                        1
Den                                       1
FIU                                       1
GA                                        1
IA                                        4
IL                                        5
IN St                                     1
JMU                                       1
LIU                                       1
NC                                        1
NE                                        1
NSULA                                     1
OK                              

In [75]:
u_v.to_csv('number_matches', index=True, sep='\t')

2nd Project: Analysis of Outside Hitters


Notes from Eduardo:
- Drop teams with a single match
- Re run the all_matches data with the new file
- Drop all players that have less than 11 attempts

RQ 1: Does the ability to win depend on how often OH receive the ball?
* Variables: % of reception attempts by OH (per team)

Hypothesis: More reception attempts by OH are associated with more sets or matches won

A list of Team names that have played more than one match

In [76]:
u_v

,Number of Matches Played
AZ,10
AZ St,10
Abilene Christian,1
Albany,1
Baylor,1
CO St,6
CU,153
Cal,9
Campbell,1
DU,1


In [77]:
teams_matches = u_v[u_v['Number of Matches Played'] > 1]

In [78]:
teams_matches

,Number of Matches Played
AZ,10
AZ St,10
CO St,6
CU,153
Cal,9
IA,4
IL,5
OK,2
OR,9
OR St,9


In [79]:
teams_list2 = teams_matches.index.tolist()
teams_list2

['AZ',
 'AZ St',
 'CO St',
 'CU',
 'Cal',
 'IA',
 'IL',
 'OK',
 'OR',
 'OR St',
 'Stan',
 'UCLA',
 'USC',
 'UT',
 'WA',
 'WA St',
 'WY',
 'Xavier']

In [80]:
# Now loop through this dataframe and calculate an average winning score.

winning_percentage = {}

for team in teams_list2:
    
    team_df = all_matches[all_matches['team'] == team]
    
    percent_won = (team_df['won_match'] == True).sum()
    tot_percent = (team_df['won_match'] == True).sum() + (team_df['won_match'] == False).sum() + (team_df['won_match'] == 'nan').sum()
    
    if percent_won == 0:
        win_perc = 0
    else:
        win_perc = int(percent_won) / int(tot_percent)
    

    winning_percentage[team] = win_perc
    
    
    

In [81]:
winning_percentage

{'AZ': 0.6988839028880451,
 'AZ St': 0.3471551419484038,
 'CO St': 0.2940708316752885,
 'CU': 0.5445432329013795,
 'Cal': 0.1663013698630137,
 'IA': 0,
 'IL': 0.16206896551724137,
 'OK': 0,
 'OR': 0.56690181565861,
 'OR St': 0.10872362326299537,
 'Stan': 1.0,
 'UCLA': 0.6571387737601829,
 'USC': 0.8115032679738562,
 'UT': 0.6738814993954051,
 'WA': 0.7852156057494867,
 'WA St': 0.8987243310516491,
 'WY': 0.5988023952095808,
 'Xavier': 0}

The below codes are checking the teams with zero and one as their percentages:

In [82]:
IA_df = all_matches[all_matches['team'] == 'IA']

In [83]:
IA_df[IA_df['won_match'] == True]

,Unnamed: 0,column_label,match_id,point_id,team,player_number,player_name,player_id,skill,skill_type,...,home_team_won,opponent_team_won,won_match,date,season,player_position,next_player_position,file_name,match_set,type_group


In [84]:
OK_df = all_matches[all_matches['team'] == 'OK']

In [85]:
OK_df[OK_df['won_match'] == True]

,Unnamed: 0,column_label,match_id,point_id,team,player_number,player_name,player_id,skill,skill_type,...,home_team_won,opponent_team_won,won_match,date,season,player_position,next_player_position,file_name,match_set,type_group


In [86]:
X_df = all_matches[all_matches['team'] == 'Xavier']

In [87]:
X_df[X_df['won_match'] == True]

,Unnamed: 0,column_label,match_id,point_id,team,player_number,player_name,player_id,skill,skill_type,...,home_team_won,opponent_team_won,won_match,date,season,player_position,next_player_position,file_name,match_set,type_group


In [88]:
Stan_df = all_matches[all_matches['team'] == 'Stan']

In [89]:
Stan_df[Stan_df['won_match'] == False]

,Unnamed: 0,column_label,match_id,point_id,team,player_number,player_name,player_id,skill,skill_type,...,home_team_won,opponent_team_won,won_match,date,season,player_position,next_player_position,file_name,match_set,type_group


In [90]:
_df = all_matches[all_matches['team'] == 'OK']

In [91]:
# Putting the winning percentages into a dataframe

winning_percent = pd.DataFrame.from_dict(winning_percentage, orient = 'index', columns = ['Matches Won'])
print(winning_percent)


        Matches Won
AZ         0.698884
AZ St      0.347155
CO St      0.294071
CU         0.544543
Cal        0.166301
IA         0.000000
IL         0.162069
OK         0.000000
OR         0.566902
OR St      0.108724
Stan       1.000000
UCLA       0.657139
USC        0.811503
UT         0.673881
WA         0.785216
WA St      0.898724
WY         0.598802
Xavier     0.000000


In [92]:
winning_percent.to_csv('Win_Percentage', index=True, sep='\t')

In [93]:
# Now loop through this dataframe and calculate an average winning score (For sets)

winning_percentage_sets = {}

for team in teams_list2:
    
    team_df = all_matches[all_matches['team'] == team]
    
    percent_won = (team_df['won_set'] == True).sum()
    tot_percent = (team_df['won_set'] == True).sum() + (team_df['won_set'] == False).sum() + (team_df['won_set'] == 'nan').sum()
    
    if percent_won == 0:
        win_perc = 0
    else:
        win_perc = int(percent_won) / int(tot_percent)
    

    winning_percentage_sets[team] = win_perc
    
    
    

In [94]:
winning_percentage_sets

{'AZ': 0.657691865147854,
 'AZ St': 0.38320179055247966,
 'CO St': 0.3967369677676084,
 'CU': 0.5450416412103283,
 'Cal': 0.2534246575342466,
 'IA': 0.1662847790507365,
 'IL': 0.38201970443349753,
 'OK': 0.16096139839766935,
 'OR': 0.5614664045333642,
 'OR St': 0.2923314462171899,
 'Stan': 0.7717528373266078,
 'UCLA': 0.6032585393740174,
 'USC': 0.5955555555555555,
 'UT': 0.6701330108827086,
 'WA': 0.6621492128678987,
 'WA St': 0.7269757311761046,
 'WY': 0.3437125748502994,
 'Xavier': 0.2744479495268139}

In [95]:
# Putting the winning percentages into a dataframe

winning_percent_set = pd.DataFrame.from_dict(winning_percentage_sets, orient = 'index', columns = ['Sets Won'])


In [96]:
winning_percent_set.to_csv('Win_Percentage_Sets', index=True, sep='\t')

In [97]:
# Counting the number of receptions for every Outside Hitter

num_receps = {}

for player, percent in left_percentages2.items():
    player_df = all_matches[all_matches['player_name'] == player]
    
    sum_reps = (player_df['skill'] == 'Reception').sum()
    
    num_receps[player] = sum_reps

In [98]:
#num_receps

In [2]:
# Putting the number of receptions into a dataframe

num_receps = pd.DataFrame.from_dict(num_receps, orient = 'index', columns = ['Number of Receptions'] )



NameError: name 'num_receps' is not defined

In [100]:
# Calculating the percentage of receptions that OH's are responsible for each team

receptions_by_OH = {}

for team in teams_list2:
    team_df = all_matches[all_matches['team'] == team]
    filtered_df = team_df[(team_df['player_position'] == 'OH') & (team_df['skill'] == 'Reception')]
    total_receps = len(team_df[team_df['skill'] == 'Reception'])
    OH_receps = len(filtered_df)
    percentage = OH_receps / total_receps
    receptions_by_OH[team] = percentage
    

In [101]:
receptions_by_OH

{'AZ': 0.48148148148148145,
 'AZ St': 0.48427672955974843,
 'CO St': 0.34243697478991597,
 'CU': 0.6345812633073101,
 'Cal': 0.35080645161290325,
 'IA': 0.4185303514376997,
 'IL': 0.32653061224489793,
 'OK': 0.515527950310559,
 'OR': 0.6008064516129032,
 'OR St': 0.48344370860927155,
 'Stan': 0.4607218683651805,
 'UCLA': 0.6644628099173554,
 'USC': 0.5315186246418339,
 'UT': 0.5363881401617251,
 'WA': 0.4121212121212121,
 'WA St': 0.4826388888888889,
 'WY': 0.3988439306358382,
 'Xavier': 0.603448275862069}

In [102]:
Receptions_by_OH = pd.DataFrame.from_dict(receptions_by_OH, orient = 'index', columns = ['Percent Receptions by OH'])

Receptions_by_OH


,Percent Receptions by OH
AZ,0.481481
AZ St,0.484277
CO St,0.342437
CU,0.634581
Cal,0.350806
IA,0.418530
IL,0.326531
OK,0.515528
OR,0.600806
OR St,0.483444


In [103]:
Receptions_by_OH.to_csv('Receptions_by_OH', index=True, sep='\t')

In [104]:
receptions = {}

winning_match = {}

winning_set = {}


for match in all_matches['match_id'].unique():
    
    match_df = all_matches[all_matches['match_id'] == match]
    
    OH_df = match_df[match_df['player_position'] == 'OH']
    
    for set_num in range(1, 6):
    
        set_df = OH_df[OH_df['set_number'] == set_num]
    
        percentage = set_df.loc[set_df['skill'] == 'Reception', 'skill'].count() / len(set_df)
        
        won_match = set_df['point'].tolist()
        won_set = set_df['won_set'].tolist()
    
        receptions[(match, set_num)] = percentage
        
        winning_match[(match, set_num)] = won_match
        
        winning_set[(match, set_num)] = won_set








In [105]:
#winning_match

In [106]:
len(winning_set)

765

In [107]:
#receptions

In [108]:
# Making this dictionary into a dataframe

receptions_new = {}

for key, value in receptions.items():
    match_id, set_number = key
    receptions_new.setdefault('match_id', []).append(match_id)
    receptions_new.setdefault('set_number', []).append(set_number)
    receptions_new.setdefault('recep_percentage', []).append(value)

In [109]:
receps_df = pd.DataFrame.from_dict(receptions_new)
receps_df

,match_id,set_number,recep_percentage
0,24fe712b9d553f6a88347598f84a2616,1,0.207921
1,24fe712b9d553f6a88347598f84a2616,2,0.250000
2,24fe712b9d553f6a88347598f84a2616,3,0.215190
3,24fe712b9d553f6a88347598f84a2616,4,0.207921
4,24fe712b9d553f6a88347598f84a2616,5,NaN
...,...,...,...
760,e49e03d1200e03589a0c88360b474504,1,0.192771
761,e49e03d1200e03589a0c88360b474504,2,0.198113
762,e49e03d1200e03589a0c88360b474504,3,0.269231
763,e49e03d1200e03589a0c88360b474504,4,0.192308


In [110]:
len(receptions)

765

Hypothesis 2: More contact with the ball by OH are associated with a lower kill % (fewer terminal contacts per set or match)

independent variable: # of receptions and attacks
dependent variable: kill percentage by OH

kill percentage = # / all other codes

Make a csv of percent of reception and attacks by OH for every set, a kill percentage by OH for every set, and label each set with each match, set outcome, and match outcome.

In [ ]:
receptions_attacks = {}




for match in all_matches['match_id'].unique():
    
    match_df = all_matches[all_matches['match_id'] == match]
    
    OH_df = match_df[match_df['player_position'] == 'OH']
    
    for set_num in range(1, 6):
    
        set_df = OH_df[OH_df['set_number'] == set_num]
    
        percentage = set_df.loc[set_df['skill'] == 'Reception', 'skill'].count() +  set_df.loc[set_df['skill'] == 'Attack', 'skill'].count()  / len(set_df)
        
     
    
        receptions_attacks[(match, set_num)] = percentage
        
     

In [ ]:
#receptions_attacks

In [ ]:
len(receptions_attacks)

In [ ]:
# Making this dictionary into a dataframe

receptions_attacks_new = {}

for key, value in receptions_attacks.items():
    match_id, set_number = key
    receptions_attacks_new.setdefault('match_id', []).append(match_id)
    receptions_attacks_new.setdefault('set_number', []).append(set_number)
    receptions_attacks_new.setdefault('recep_attack_percentage', []).append(value)
    
    

In [ ]:
receps_attack_df = pd.DataFrame.from_dict(receptions_attacks_new)
receps_attack_df

In [ ]:
kill_percent = {}




for match in all_matches['match_id'].unique():
    
    match_df = all_matches[all_matches['match_id'] == match]
    
    OH_df = match_df[match_df['player_position'] == 'OH']
    
    for set_num in range(1, 6):
    
        set_df = OH_df[OH_df['set_number'] == set_num]
    
        percentage = set_df.loc[set_df['evaluation_code'] == '#', 'evaluation_code'].count()  / len(set_df)
        
     
    
        kill_percent[(match, set_num)] = percentage
        

In [ ]:
#kill_percent

In [ ]:
match_1 = all_matches[all_matches['match_id'] == '24fe712b9d553f6a88347598f84a2616']

OH_1 = match_1[match_1['player_position'] == 'OH']

set_1 = OH_1[OH_1['set_number'] == 1]


#set_1

set_1.loc[set_1['evaluation_code'] == '#', 'evaluation_code'].count()


In [ ]:
len(set_1)

In [ ]:
18/101

In [ ]:
len(kill_percent)

In [ ]:
# Making this dictionary into a dataframe

kill_percent_new = {}

for key, value in kill_percent.items():
    match_id, set_number = key
    kill_percent_new.setdefault('match_id', []).append(match_id)
    kill_percent_new.setdefault('set_number', []).append(set_number)
    kill_percent_new.setdefault('kill_percentage', []).append(value)
    
    
    
    

In [ ]:
#kill_percent_new

In [ ]:
kill_percent1 = pd.DataFrame.from_dict(kill_percent_new)
kill_percent1

In [ ]:
receps_df

In [ ]:
receps_attack_df

In [ ]:
receps_df['kill_percentage'] = kill_percent1['kill_percentage']

receps_df['recep_attack_perc'] = receps_attack_df['recep_attack_percentage']

In [ ]:
receps_df

In [ ]:
receps_df.to_csv('vball_df2', index=True, sep='\t')

In [ ]:
# Need receptions and attacks to be a count- not a percentage

receps_attack_count = {}




for match in all_matches['match_id'].unique():
    
    match_df = all_matches[all_matches['match_id'] == match]
    
    OH_df = match_df[match_df['player_position'] == 'OH']
    
    for set_num in range(1, 6):
    
        set_df = OH_df[OH_df['set_number'] == set_num]
    
        count = set_df.loc[set_df['skill'] == 'Reception', 'skill'].count() +  set_df.loc[set_df['skill'] == 'Attack', 'skill'].count()
        
     
    
        receps_attack_count[(match, set_num)] = count

In [ ]:
#receps_attack_count

In [ ]:
# Making this dictionary into a dataframe

receps_attack_new = {}

for key, value in receps_attack_count.items():
    match_id, set_number = key
    receps_attack_new.setdefault('match_id', []).append(match_id)
    receps_attack_new.setdefault('set_number', []).append(set_number)
    receps_attack_new.setdefault('recep_attack_count', []).append(value)
    
    

In [ ]:
receps_attack_count = pd.DataFrame.from_dict(receps_attack_new)
receps_attack_count

In [ ]:
receps_df

In [ ]:
receps_df['recep_attack_count'] = receps_attack_count['recep_attack_count']

In [ ]:
receps_df

In [ ]:
receps_df.to_csv('vball_df3', index=True, sep='\t')

For research question number 3, we want the number of receptions and attacks by OH in proportion to their teams receptions and attacks.

need to find the number of receptions and attacks by OH / number of reception and attacks by whole team

In [ ]:
teams_list2

In [ ]:
receps3 = {}


for team in teams_list2:
    
    team_df = all_matches[all_matches['team'] == team]
    
    for match in team_df['match_id'].unique():
        
        match_df = team_df[team_df['match_id'] == match]
        
        for set_num in match_df['set_number'].unique():
            
            set_df = match_df[match_df['set_number'] == set_num]
            
            OH_df = set_df[set_df['player_position'] == 'OH']
            
            oh_rec =  OH_df['skill'].eq('Reception').sum()
            
            tot = set_df['skill'].eq('Reception').sum()
            
            percent = oh_rec / tot
            
            set_val = set_df.iloc[0]['won_set']
            match_val = set_df.iloc[0]['won_match']
            
            
            receps3[(team, match, set_num, set_val, match_val)] = percent
        
        

In [ ]:
#receps3

In [ ]:
receps4 = {}

for key, value in receps3.items():
    team, match_id, set_number, set_val, match_val = key
    receps4.setdefault('team', []).append(team)
    receps4.setdefault('match_id', []).append(match_id)
    receps4.setdefault('set_number', []).append(set_number)
    receps4.setdefault('won_set', []).append(set_val)
    receps4.setdefault('won_match', []).append(match_val)
    receps4.setdefault('recep_percent_OH', []).append(value)

In [ ]:
receps_df4 = pd.DataFrame.from_dict(receps4)
receps_df4

In [ ]:
receps_df4.to_csv('recept_perc_OH', index=True, sep='\t')

In [ ]:
OH_proportions = {}

for key, val in outside_hitters3.items():
    
    
    team = all_matches.loc[all_matches['player_name'] == key, 'team'].values[0]
    
    team_df = all_matches[all_matches['team'] == team]
    
    team_count1 = team_df['skill'].value_counts()['Attack']
    
    team_count2 = team_df['skill'].value_counts()['Reception']
    
    team_count = team_count1 + team_count2
    
    player_df = all_matches[all_matches['player_name'] == key]
    
    player_count1 = player_df['skill'].value_counts()['Attack']
    
    player_count2 = player_df['skill'].eq('Reception').sum()
                                                      
    player_count = player_count1 + player_count2
    
    proportion = player_count / team_count
    
    OH_proportions[(key, team)] = proportion
    
    
    
    

In [ ]:
#OH_proportions

In [ ]:
len(OH_proportions)

In [ ]:
OH_prop = {}

for key, value in OH_proportions.items():
    player_name, team = key
    OH_prop.setdefault('player_name', []).append(player_name)
    OH_prop.setdefault('team', []).append(team)
    OH_prop.setdefault('OH_proportion', []).append(value)

In [ ]:
#OH_prop

In [ ]:
OH_prop_df = pd.DataFrame.from_dict(OH_prop)

In [ ]:
#OH_prop_df

In [ ]:
filtered_OH_prop = OH_prop_df[OH_prop_df['team'].isin(teams_list2)]

In [ ]:
#filtered_OH_prop

In [ ]:
filtered_OH_prop.to_csv('OH_proportion2', index=True, sep='\t')

In [ ]:
receptions_attacks2 = {}
receptions_attacks3 = {}



for match in all_matches['match_id'].unique():
    
    match_df = all_matches[all_matches['match_id'] == match]
    
    OH_df = match_df[match_df['player_position'] == 'OH']
    
    
    for set_num in range(1, 6):
    
        setOH_df = OH_df[OH_df['set_number'] == set_num]
        
        set_df = match_df[match_df['set_number'] == set_num]
    
        percentage_recep = setOH_df.loc[setOH_df['skill'] == 'Reception', 'skill'].count()  / len(set_df)
        percentage_attack = setOH_df.loc[setOH_df['skill'] == 'Attack', 'skill'].count() / len(set_df)
     
    
        receptions_attacks2[(match, set_num)] = percentage_recep
        
        
        receptions_attacks3[(match, set_num)] = percentage_attack
        
     

In [ ]:
#receptions_attacks2

In [ ]:
#receptions_attacks3

In [ ]:
receptions_attacks2 = {}
receptions_attacks3 = {}



for match in all_matches['match_id'].unique():
    
    match_df = all_matches[all_matches['match_id'] == match]
    
    OH_df = match_df[match_df['player_position'] == 'OH']
    
    
    for set_num in range(1, 6):
    
        setOH_df = OH_df[OH_df['set_number'] == set_num]
        
        set_df = match_df[match_df['set_number'] == set_num]
    
        percentage_recep = setOH_df.loc[setOH_df['skill'] == 'Reception', 'skill'].count()  / len(set_df)
        percentage_attack = setOH_df.loc[setOH_df['skill'] == 'Attack', 'skill'].count() / len(set_df)
     
    
        receptions_attacks2[(match, set_num)] = percentage_recep
        
        
        receptions_attacks3[(match, set_num)] = percentage_attack

In [ ]:
oh_receps4 = {}
oh_attack4 = {}

for hitter in Outside_Hitters:
    
    hitter_df = all_matches[all_matches['player_name'] == hitter]
    
    percentage_recep = hitter_df.loc[hitter_df['skill'] == 'Reception', 'skill'].count()  / len(hitter_df)
    
    percentage_attack = hitter_df.loc[hitter_df['skill'] == 'Attack', 'skill'].count() / len(hitter_df)
    
    row = hitter_df.loc[hitter_df['player_name'] == hitter]
    
    team = row['team'].values[0]
        
    oh_receps4[(hitter, team)] = percentage_recep
    
    oh_attack4[(hitter, team)] = percentage_attack
        
        
        
    

In [ ]:
#oh_receps4

In [ ]:
OH_recep5 = {}

for key, value in oh_receps4.items():
    player_name, team = key
    OH_recep5.setdefault('player_name', []).append(player_name)
    OH_recep5.setdefault('team', []).append(team)
    OH_recep5.setdefault('OH_receps', []).append(value)

In [ ]:
OH_attack5 = {}

for key, value in oh_attack4.items():
    player_name, team = key
    OH_attack5.setdefault('player_name', []).append(player_name)
    OH_attack5.setdefault('team', []).append(team)
    OH_attack5.setdefault('OH_attacks', []).append(value)

In [ ]:
oh_receps5 = pd.DataFrame.from_dict(OH_recep5)

In [ ]:
oh_attacks5 = pd.DataFrame.from_dict(OH_attack5)

In [ ]:
#oh_attacks5

In [ ]:
#oh_receps5

In [ ]:
col4 = oh_attacks5['OH_attacks']

In [ ]:
merged_df = pd.concat([oh_receps5, col4], axis=1)

In [ ]:
#merged_df

In [ ]:
merged_df.to_csv('oh_perc', index=True, sep='\t')

In [115]:
all_matches['evaluation_code'].unique()

array([nan, '!', '#', '-', '/', '+', '='], dtype=object)